In [296]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits

In [315]:
def unique_and_count(df, col="TIC"):
    unique = df[col].unique().astype("<U12")
    return unique, len(unique)

In [316]:
# Composite Table of confirmed exoplanets
path = "../data/2022_07_27_input_catalog_star_planet_systems.csv"

print(f"[UP] Using compiled input catalog from {path}")

input_catalog = pd.read_csv(path) 

[UP] Using compiled input catalog from ../data/2022_07_27_input_catalog_star_planet_systems.csv


In [317]:
vetted_table = pd.read_csv("../results/2022_07_flares_vetted.csv")
unvetted_table = pd.read_csv("../results/2022_07_flares.csv")
vetted_table.shape, unvetted_table.shape

((10275, 22), (10745, 22))

In [318]:
unique_input_catalog, n_unique_input_catalog = unique_and_count(input_catalog,col="TIC")
assert n_unique_input_catalog == 2993
assert unique_and_count(input_catalog,col="hostname")[1] == 2949 # 2948 + 1 NaN


In [319]:
unique_in_vetted_table, n_unique_in_vetted_table = unique_and_count(vetted_table)


In [320]:
unique_in_unvetted_table, n_unique_in_unvetted_table = unique_and_count(unvetted_table)

In [321]:
nolc = pd.read_csv("../results/2022_07_nolc.txt").drop_duplicates()
# nolc.TIC = nolc.TIC.str[4:]
unique_nolc, n_unique_nolc = unique_and_count(nolc)
n_unique_nolc

1336

In [322]:
nothingfound = pd.read_csv("../results/2022_07_listed_but_nothing_found.csv")
nothingfound.TIC = nothingfound.TIC.str[4:]

unique_nothingfound, n_unique_nothingfound = unique_and_count(nothingfound)

In [323]:
assert n_unique_in_vetted_table == n_unique_in_unvetted_table

In [324]:
lost_tics = list(set(unique_input_catalog) -
                 set(unique_in_vetted_table) - 
                 set(unique_nolc) -
                 set(unique_nothingfound))

In [325]:
n_unique_input_catalog, n_unique_in_vetted_table, n_unique_nolc, n_unique_nothingfound

(2993, 1522, 1336, 414)

In [326]:
len(lost_tics)

22

In [327]:
from lightkurve import search_lightcurve

In [328]:
lcs  = search_lightcurvefile("TIC 26657021")   
conditions = (lcs.exptime.value < 130) & (lcs.author != "TASOC")
lcs_sel = lcs[conditions]

len(lcs_sel)

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/ipykernel_launcher.py:1: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  """Entry point for launching an IPython kernel.


0

In [329]:
lcs  = search_lightcurve(f"TIC {lost_tics[0]}")

In [330]:
lcs

#,mission,year,author,exptime,target_name,distance
,,,,s,,arcsec
0,TESS Sector 37,2021,SPOC,20,287331845,0.0
1,TESS Sector 37,2021,SPOC,120,287331845,0.0
2,TESS Sector 37,2021,TESS-SPOC,600,287331845,0.0


In [312]:
lost_tics[0]

'187309502'

In [314]:
for tic in lost_tics:

    lcs  = search_lightcurve(f"TIC {tic}")
    try:
        conditions = (lcs.exptime.value < 130) & (lcs.author != "TASOC")  & (lcs.author != "K2")
        lcs_sel = lcs[conditions]
        print(len(lcs_sel))
        if len(lcs_sel)==0:
            with open("../results/2022_07_nolc.txt","a") as f:
                f.write(tic + "\n")
            with open("../results/2022_07_k2_short_cadence.txt","a") as f:
                f.write(tic + "\n")
    except KeyError:
        with open("../results/2022_07_error_in_search.txt","a") as f:
            f.write(tic + "\n")

0
2
0
4
0
0
1
4
2
0
2
3
3
0
0
2
0
0
0
0
1
1
3
0
1
2
0
0
2
2
0
1
0
2
0
0
0
0
0
1
2
1
1
0
0
